In [102]:
import numpy as np
from numpy import sqrt
from numpy import sin
from numpy import pi
from numpy import cos
from numpy import exp
from numpy import tanh
from numpy import zeros
from numpy import arccos
from numpy import log10
from matplotlib import pyplot as plt
import scipy.optimize as opt
import emcee
from pylab import plot
from scipy import integrate
import corner
import random
from collections import Counter
import time
import scipy
from scipy.stats import poisson # use as poisson.pmf(number of events , mean value)
from scipy import optimize
import multiprocessing as mp
from multiprocessing import Pool

import astropy.units as u
import astropy.constants as c
from astropy.cosmology import FlatLambdaCDM, z_at_value
from tqdm import *
from astropy.cosmology import Planck13 as cosmo
from astropy import constants as const

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cmasher as cmr
import matplotlib.colors as clrs
import matplotlib.cm as cmap

colors = ['#8a1f1f','#79a43a','#C59D34','#171782', '#cf6717','#ad6faa',
          '#009999','#828282']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=colors)
plt.rcParams['lines.linewidth'] = 3.0
plt.rcParams['axes.linewidth'] = 2.

# Some useful quantities

In [103]:
import cmath
# we work in geometrized units
Msol = 1.47680000000000000000000000000000000 # in km
c = 1 # we have actually put them to one everywhere in the code already
G = 1
year = 60*60*24*(365.250000000000000000000000000); # s
cc = 2.9980000000000000000000000000000*10**5 # velocity of light in Km/s
pc = 3.0856770000000000000000000000*10**13 # km

H0 = 67.9 # km /s / Mpc # changed to PLANCK 2015
Omegam = 0.306
OmegaL = 0.694
Omegar = 0.

# some useful functions 
def distL(z):
    return (1+z) * integrate.quad(invE,0,z)[0] *cc / H0 *10**6*pc # km

def Ez(z):
    return sqrt(Omegar*(1+z)**4+Omegam*(1+z)**3+OmegaL)

def invE(z):
    return 1. / Ez(z)

def primopezzo(zp):
    return 1/Ez(zp)*(integrate.quad(invE,0,zp)[0])**2

def dVdz(z):
    return 4*pi*(cc/H0)**3*primopezzo(z) # Mpc**3

def dtdf(f,m1,m2):  # Hz, km, km, gravitational wave frequency
    return 5/(96*pi**(8/3))*(1./cc*(m1 + m2)*(m1*m2/(m1 + m2)**2)**(3/5))**(-5/
  3)*f**(-11/3)  # s^2, assuming f in Hz and m in km

def solof(f):
    return f**(-11/3)

def nu(m1,m2):
    return m1*m2/((m1+m2)**2)

def Mc(m1,m2):
    return (m1 + m2) * (m1*m2/((m1+m2)**2))**(3/5)

In [188]:
# Some other useful functions for frequencies, time of merger, ecc 

# frequency functions 

def fMIN2(fmax0,m1,m2,Tobs): # gravitational wave frequency
    return 1/(1/fmax0**(8/3)+256/5*(Mc(m1,m2)/cc)**(5/3)*pi**(8/3)*(Tobs))**(3/8)

def fmax(m1,m2,fmin,Tobs): # gravitational wave frequency
    return fmin*((5*cc)/(5*cc-256*fmin**(8/3)*pi**(8/3)*Tobs*Mc(m1,m2)*(Mc(m1,m2)/cc)**(2/3)))**(3/8)


# gives fmax, given fmin and Tobs. If fmax is above 10 Hz, gives 10 Hz instead. frequency is always in Hz

def getfmax(m1,m2,fmin,Tobs): # gravitational wave frequency
    if fMIN2(1,m1,m2,Tobs)>fmin:
        return fmax(m1,m2,fmin,Tobs)
    else: 
        return 1

def Tmerger(m1,m2,fmin): # gravitational wave frequency
    return 5. * (m1 + m2)**(1./3.) * cc**(5./3.)/ (256. * fmin**(8./3.) * m1 * m2 * np.pi**(8./3.))



from scipy.optimize import root_scalar
def findFmin(timemerger,m1,m2):
    def condition(fmin):
        return timemerger-Tmerger(m1,m2,fmin)
    return root_scalar(condition,bracket=(10**-4,1)).root



In [192]:
findFmin(0.001 * year, 10 ,12)

0.9679201698698369

In [351]:
#here we want to add also the part with the SNR 

# LISA noise parameters
Amp = 5/10*18/10*10**-44
alpha = 138/1000
beta = -221
kappa = 521
gamma = 1680
fk = 113/10**5
L0 = 25/10*10**9
fstar = 1909/100*10**-3
pm = 10**-12

skyav=(4/5*sqrt(2)*sin(pi/3))

# noise function, checked with Mathematica

def Sacc(f):
    return (3*10**-15)**2*(1+(4/10*10**-3/f)**2)*(1+(f/(8*10**-3))**4)

def Sgal(f):
    return Amp*f**(-7/3)*exp(-f**alpha+beta*f*sin(kappa*f))*(1+tanh(gamma*(fk-f)))

def Soms(f):
    return (15. *pm)**2*(1+(2*10**-3/f)**4)  # noi: 10

def Sacc(f):
    return (3*10**-15)**2*(1+(4/10*10**-3/f)**2)*(1+(f/(8*10**-3))**4)
    
def SnSA(f):
    return 10/3*(4*Sacc(f)/(2*pi*f)**4+Soms(f))/(L0**2)*(1+6/10*(f/fstar)**2)

def R(f):
    return 3/20*2

def Sn(f):
    return abs(SnSA(f)+Sgal(f))

def Pn(f): # gravitational wave frequency
    return abs(Sn(f)*R(f))

def Aplus(iota,psi):
    return -(1+cos(iota)**2)*cos(2*psi)-2*cos(iota)*sin(2*psi)

def Across(iota,psi):
    return (1+cos(iota)**2)*sin(2*psi)-2*cos(iota)*cos(2*psi)

def Fplus(theta,phi):
    return 1/2*(1+cos(theta)**2)*cos(2*phi)

def Fcross(theta,phi):
    return cos(theta)*sin(2*phi)

def factorsky(iota,psi,theta,phi):
    return Fplus(theta,phi)*Aplus(iota,psi)+1j*Fcross(theta,phi)*Across(iota,psi)

def factorskySNR(iota,psi,theta,phi):
    return abs(sqrt(2)*sin(pi/3)*(Fplus(theta,phi)*Aplus(iota,psi)+1j*Fcross(theta,phi)*Across(iota,psi)))



# Waveform and SNR

In [352]:
# waveform

def ampl(m1,m2,d): 
    return (Mc(m1,m2)*G)**(5/6)*sqrt(5/24)/(pi**(2/3)*d*c**(3/2))
def habs(m1,m2,d,f): # gravitational wave frequency
    return ampl(m1,m2,d)*f**(-7/6)

#print(habs(12.4913,12.3884,10.8287*10**6*pc,1./cc))

In [353]:
# Here we define the SNR; 
# be careful that I have already put the factor for the sky average
# we use the GW only template.

def SNR(iota,psi,theta,phi,fmin,fmax,m1,m2,d): # gravitational wave frequency
    return sqrt(4*integrate.quad(lambda x: (factorskySNR(iota,psi,theta,phi)*habs(m1,m2,d, x/cc))**2/(Pn(x)*(cc**2)), fmin, fmax)[0])
#different noise here!

def SNRAverage(fmin,fmax,m1,m2,d): # gravitational wave frequency
    return sqrt(4*integrate.quad(lambda x: (skyav*habs(m1,m2,d, x/cc))**2/(Pn(x)*(cc**2)), fmin, fmax)[0])


In [11]:
z = 0.24716227944027847
tmerger = 5.073762956477287*year
m1 = 88.7
m2 = 64.93
(iota, psi, theta, phi) = 2.23353428920549, 5.484427462941964, 2.014813251983311, 2.374251372635906
fmin = findFmin(tmerger,m1,m2)
#print(fmin)
tobs = 6.*year
fmax= getfmax(m1*(1.+z),m2*(1.+z),fmin/(1+z),tobs)
#print(fmax)
loSNR=SNR(iota,psi,theta,phi,fmin/(1+z),fmax,m1*(1+z),m2*(1+z),distL(z))
print(loSNR) # 6.155,9.07

9.07125245940824


# Mass data

In [12]:
eobdata = np.loadtxt('EOBmasses.dat')
NRdata = np.loadtxt('NRmasses.dat')
Phendata = np.loadtxt('PHENmasses.dat')

In [45]:
import h5py

data = h5py.File('/Users/andreacaputo/Desktop/Phd/BinaryLISALIGO/Event_SamplesWaveTransient/all_events/o1o2o3_mass_c_iid_mag_two_comp_iid_tilt_powerlaw_redshift_mass_data.h5', 'r')
mass_ppd = data["ppd"]
mass_lines = data["lines"]

# Create a flat copy of the array
flat = mass_ppd[:].flatten() /  np.sum(mass_ppd[:].flatten())

 
mass_1 = np.linspace(3, 100, 1000)
mass_ratio = np.linspace(0.1, 1, 500)

ratePL = np.trapz(np.trapz(mass_ppd, mass_ratio, axis=0), mass_1, axis = 0)

In [75]:
mass_ppd[:].shape

(500, 1000)

In [67]:
mass_lines.keys()

<KeysViewHDF5 ['mass_1', 'mass_ratio']>

In [82]:
mass_lines['mass_1'][:].shape

(10000, 1000)

In [89]:
np.random.choice(len(mass_lines['mass_ratio'][:]))
try1 = np.outer(mass_lines['mass_ratio'][:][5166], mass_lines['mass_1'][:][5166] )
flat = mass_ppd[:].flatten() /  np.sum(mass_ppd[:].flatten())


5166

In [96]:
try1.flatten()

array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
       1.53344045e-110, 7.08344558e-111, 3.26828825e-111])

In [93]:
try1 = np.outer(mass_lines['mass_ratio'][:][5166], mass_lines['mass_1'][:][5166] )

In [86]:
mass_lines['mass_ratio'][:].shape

(10000, 500)

In [83]:
massratioAll = mass_lines['mass_ratio'][:]
mass1All = mass_lines['mass_1'][:]

In [85]:
np.einsum('ij, ik -> jk', massratioAll , mass1All).shape

(500, 1000)

# Sampling 

In [147]:
def gendistrNewGamma(N,iteration,massoption,SNRth, Tobs, duty):
    

    N=int(N)
    # Define functions to store intermediate products; I haven't decided on the most convenient format for the final catalogs yet...

    massmin=50*Msol
    massmax=100*Msol

    # Largest horizon redshift; don't waste computing time above this
    zmax = 0.512
    zmin = 0.1
    tmax = 40
    tobs = Tobs # in years
    # ground-based duty cycle
    dutycycle= duty

    data=[]
    for i in range(N):

        z = np.random.uniform(zmin,zmax)
        dl = distL(z)
        
        if massoption == 'EOB':
            index = np.random.choice(len(eobdata))
            m1 = Msol*eobdata[index][0]
            m2 = Msol*eobdata[index][1]
            Rate = 0.13*10**(-9) #19.*10**(-9)
        
        if massoption == 'PHEN':
            index = np.random.choice(len(Phendata))
            m1 = Msol* Phendata[index][0]
            m2 = Msol*Phendata[index][1]
            Rate = 0.13*10**(-9) #19.*10**(-9)
            
            
        if massoption == 'NR':
            index = np.random.choice(len(NRdata))
            m1 = Msol*NRdata[index][0]
            m2 = Msol*NRdata[index][1]
            Rate = 0.13*10**(-9) #19.*10**(-9)
                
        # Sky-location, inclination, polarization, initial phase
        cosiota = random.uniform(-1.,1.)
        psi = random.uniform(0,2*pi)
        costheta=random.uniform(-1.,1.)
        phi=random.uniform(0,2*pi)
        iota=np.arccos(cosiota)
        theta=np.arccos(costheta)
        #phic = np.random.uniform(0,2.*np.pi)
        
        # Merger time
        tmerger=np.random.uniform(1*year,41*year)
        
        #fmin=np.random.uniform(1e-5,0.01)
        fmin=findFmin(tmerger,m1,m2)
        Fmax= getfmax(m1*(1+z),m2*(1+z),fmin/(1+z),tobs*year)  #fmax(m1,m2,fmin,tobs*year)
        #Fmax = 1 #Fmax=1
        snr=np.sqrt(dutycycle)*SNR(iota,psi,theta,phi,fmin/(1+z),Fmax,m1*(1+z),m2*(1+z),dl)

        dVcdz = cosmo.comoving_volume(z).value 
        rateSampling = np.random.gamma(1, 1.46*Rate) #np.random.normal(Rate, 0.3e-9)#1 / np.random.gamma(1, 1/ Rate)
        
        integralbulk = rateSampling * tmax * (zmax-zmin) * dVcdz   *np.heaviside(snr-SNRth,0) * (1./(1.+z))

        data.append(np.array([m1,m2,z,fmin,integralbulk,snr]))

    return np.array(data).T



In [286]:
prova= np.array([np.random.gamma(1.9, 0.65*0.13) for n in range(10000)])

np.percentile(prova, 95)

0.3807312160835836

In [287]:
np.percentile(prova, 5)

0.02732035886465833

In [278]:
np.median(prova)

0.12889218044858736

In [220]:
np.percentile(prova, 5)

0.009670233208198336

In [148]:
def consolidatedistrGamma(Nsingle,iterations,massoption,SNRth, Tobs, duty):

    Nsingle = int(Nsingle)
    iterations = int(iterations)

    Ntot = Nsingle*iterations
    data=[]
    for it in range(0,iterations):
        data.append(gendistrNewGamma(Nsingle,it,massoption,SNRth, Tobs, duty))

    data=np.array(data)

    #[m1,m2,z,pdetLIGO,pdetCE,tmerger,SNR4,SNR10,integralbulk]

    m1 = np.concatenate(data[:,0])
    m2 = np.concatenate(data[:,1])
    z = np.concatenate(data[:,2])
    #pdetLIGO = np.concatenate(data[:,3])
    #pdetCE = np.concatenate(data[:,4])
    #tmerger = np.concatenate(data[:,3])
    fmin = np.concatenate(data[:,3])
    #SNR4 = np.concatenate(data[:,6])
    #SNR10 = np.concatenate(data[:,7])
    integralbulk = np.concatenate(data[:,4])
    SNR10 = np.concatenate(data[:,5])

    return m1,m2,z,fmin,integralbulk,SNR10

In [149]:
def NeventsGamma(bigdata,massoption):

    # bigdata is the output of consolidatedistr
    m1,m2,z,fmin,integralbulk,SNR10 = bigdata #consolidatedistr(Nsingle,iterations)
    Ntot = len(m1) 
    montecarlo_contributions = integralbulk / Ntot
        
    return np.sum(montecarlo_contributions)
    


In [151]:
print(NeventsGamma(consolidatedistrGamma(10**5,1,'EOB',8, 10, 0.75),'EOB'),NeventsGamma(consolidatedistrGamma(10**4,1,'EOB',8,  10, 0.75),'EOB')     
)

print(NeventsGamma(consolidatedistrGamma(10**5,1,'NR',8,  10, 0.75),'NR'),NeventsGamma(consolidatedistrGamma(10**4,1,'NR',8,  10, 0.75),'NR')     
)

print(NeventsGamma(consolidatedistrGamma(10**5,1,'PHEN',8,  10, 0.75),'PHEN'),NeventsGamma(consolidatedistrGamma(10**4,1,'PHEN',8,  10, 0.75),'PHEN')     
)

8.669906375972902 8.555023530228894
7.56036695902468 7.64312226169843
7.510186572853788 7.7525176269192


In [141]:
print(NeventsGamma(consolidatedistrGamma(10**4,1,'NR',8, 10, 0.75),'NR'),NeventsGamma(consolidatedistrGamma(10**4,1,'NR',8, 10, 0.75),'NR')     
)

2.566863913282771 2.685090915347251


In [140]:
print(NeventsGamma(consolidatedistrGamma(10**4,1,'PHEN',8, 6, 0.75),'PHEN'),NeventsGamma(consolidatedistrGamma(10**4,1,'PHEN',8, 6, 0.75),'PHEN')     
)

2.608632067395162 2.586479613145505


# Old sampling, no Gamma function for the rate. Flat distribution in the interval given by the collaboration 

In [155]:
def gendistrOld(N,iteration,massoption,SNRth, Tobs, duty):
    

    N=int(N)
    # Define functions to store intermediate products; I haven't decided on the most convenient format for the final catalogs yet...

    massmin=50*Msol
    massmax=100*Msol

    # Largest horizon redshift; don't waste computing time above this
    zmax = 0.512
    zmin = 0.1
    # Largest comoving distance for sampling
    #??? dove la usa? -->  cdmax=astropy.cosmology.Planck15.comoving_distance(zmax).value # Mpc
    # z pdf normalization. Comoving volume at largest redshift (because comoving volume at z=0 is 0)
    # Vczhor = (astropy.cosmology.Planck15.comoving_volume(zmax)/astropy.units.Gpc**3).decompose().value #Gpc^3
    # largest merger time (yrs)
    tmax = 40
    tobs = Tobs
    # ground-based duty cycle
    dutycycle= duty

    data=[]
    for i in range(N):

        #Comoving distance uniform on sphere
        #while True:
        #    cd = np.sum(np.random.uniform(0, cdmax,3)**2)**0.5
        #    if cd<cdmax: break
        # Convert to redshift
        #z = astropy.cosmology.z_at_value(astropy.cosmology.Planck15.comoving_distance,cd*astropy.units.Mpc )
        # Convert to luminosity distance


        z = np.random.uniform(zmin,zmax)
        dl = distL(z)

        # Mass spectrum
        if massoption=='log': # Log flat distribution in both masses
            bothm=10**np.random.uniform(np.log10(massmin),np.log10(massmax),2)
            m1= max(bothm)
            m2= min(bothm)
            Rate = 57*10**(-9)
        
        if massoption == 'EOB':
            index = np.random.choice(len(eobdata))
            m1 = Msol*eobdata[index][0]
            m2 = Msol*eobdata[index][1]
            Rate = 0.13*10**(-9)
        
        if massoption == 'PHEN':
            index = np.random.choice(len(Phendata))
            m1 = Msol* Phendata[index][0]
            m2 = Msol*Phendata[index][1]
            Rate = 0.13*10**(-9)
            
        if massoption == 'NR':
            index = np.random.choice(len(NRdata))
            m1 = Msol*NRdata[index][0]
            m2 = Msol*NRdata[index][1]
            Rate = 0.13*10**(-9)
            
        if massoption == 'breakPL' or massoption == 'breakPLcut':
            
            sample_index = np.random.choice(a=flat.size, p=flat)
            adjusted_index = np.unravel_index(sample_index, mass_ppd[:].shape)
            m1 = Msol * mass_1[adjusted_index[1]]
            m2 = Msol * mass_1[adjusted_index[1]] * mass_ratio[adjusted_index[0]]
            
        if massoption == 'PL_error':
            index1 = np.random.choice(len(mass_lines['mass_ratio'][:]))
            mass_ppd1 = np.outer(mass_lines['mass_ratio'][:][5166], mass_lines['mass_1'][:][5166] )
            flat1 = mass_ppd1.flatten() /  np.sum(mass_ppd1.flatten())
            
            sample_index = np.random.choice(a=flat1.size, p=flat1)
            adjusted_index = np.unravel_index(sample_index, mass_ppd1.shape)
            m1 = Msol * mass_1[adjusted_index[1]]
            m2 = Msol * mass_1[adjusted_index[1]] * mass_ratio[adjusted_index[0]]

        
        if massoption=='powerlaw': # Power law with spectral index alpha in primary; uniform in secondary
            alpha=-2.3
            m1 = (massmin**(alpha+1.)+np.random.uniform(0.,1.)*(massmax**(alpha+1.)-massmin**(alpha+1.)))**(1./(alpha+1.))
            m2 = np.random.uniform(massmin,m1)
            Rate = 0.13*10**(-9)
                
        # Sky-location, inclination, polarization, initial phase
        cosiota = random.uniform(-1.,1.)
        psi = random.uniform(0,2*pi)
        costheta=random.uniform(-1.,1.)
        phi=random.uniform(0,2*pi)
        iota=np.arccos(cosiota)
        theta=np.arccos(costheta)
        #phic = np.random.uniform(0,2.*np.pi)
        
        # Merger time
        tmerger=np.random.uniform(1*year,41*year)
        
        #fmin=np.random.uniform(1e-5,0.01)
        fmin=findFmin(tmerger,m1,m2)
        Fmax= getfmax(m1*(1+z),m2*(1+z),fmin/(1+z),tobs*year)  #fmax(m1,m2,fmin,tobs*year)
        #Fmax = 1 #Fmax=1
        snr=np.sqrt(dutycycle)*SNR(iota,psi,theta,phi,fmin/(1+z),Fmax,m1*(1+z),m2*(1+z),dl)


        dVcdz = cosmo.comoving_volume(z).value 
        
        if massoption == 'breakPLcut':
            
            integralbulk = tmax * (zmax-zmin) * dVcdz   *np.heaviside(snr-SNRth,0) * np.heaviside(m1- 45 * Msol,0) * (1./(1.+z))

        else:
            
            integralbulk = tmax * (zmax-zmin) * dVcdz   *np.heaviside(snr-SNRth,0) * (1./(1.+z))

            
        data.append(np.array([m1,m2,z,fmin,integralbulk,snr]))

    return np.array(data).T


In [156]:
def consolidatedistrOld(Nsingle,iterations,massoption,SNRth, Tobs, duty):

    Nsingle = int(Nsingle)
    iterations = int(iterations)

    Ntot = Nsingle*iterations
    data=[]
    for it in range(0,iterations):
        data.append(gendistrOld(Nsingle,it,massoption,SNRth, Tobs, duty))

    data=np.array(data)

    #[m1,m2,z,pdetLIGO,pdetCE,tmerger,SNR4,SNR10,integralbulk]

    m1 = np.concatenate(data[:,0])
    m2 = np.concatenate(data[:,1])
    z = np.concatenate(data[:,2])
    #pdetLIGO = np.concatenate(data[:,3])
    #pdetCE = np.concatenate(data[:,4])
    #tmerger = np.concatenate(data[:,3])
    fmin = np.concatenate(data[:,3])
    #SNR4 = np.concatenate(data[:,6])
    #SNR10 = np.concatenate(data[:,7])
    integralbulk = np.concatenate(data[:,4])
    SNR10 = np.concatenate(data[:,5])

    return m1,m2,z,fmin,integralbulk,SNR10

In [157]:
def NeventsOld(bigdata,massoption,rateinterval):

    # bigdata is the output of consolidatedistr
    m1,m2,z,fmin,integralbulk,SNR10 = bigdata #consolidatedistr(Nsingle,iterations)

    Ntot = len(m1)


    # Intrisinc merger rate from LIGO O2 catalog. Use the numbers reported in Sec 4 of 1811.12940, which averages over the two pipelines.
    totalrate={}
    if massoption=='powerlaw':
        totalrate['median'] =  57*10**(-9)
        totalrate['upper'] = (57 + 40)*10**(-9)
        totalrate['lower'] = (57. - 25.)*10**(-9)
    if massoption=='log':
        totalrate['median'] = 0.13*10**(-9) #19.*10**(-9)
        totalrate['upper'] = (19. + 13.)*10**(-9)
        totalrate['lower'] = (19. - 8.2)*10**(-9)
        
    if massoption=='EOB' or massoption=='NR' or massoption=='PHEN':
        totalrate['median'] = 0.13*10**(-9) #19.*10**(-9)
        totalrate['upper'] = (0.13 + 0.3)*10**(-9)
        totalrate['lower'] = (0.13 - 0.11)*10**(-9)
        
    if massoption == 'breakPL' or massoption == 'breakPLcut' or 'PL_error':
        totalrate['median'] = ratePL *10**(-9)

    
    montecarlo_contributions = totalrate[rateinterval] * integralbulk / Ntot
    

    return np.sum(montecarlo_contributions)



In [158]:
NeventsOld(consolidatedistrOld(10**4,1,'breakPLcut',8, 10, 0.75),'breakPLcut','median')

15.642078806972979

In [159]:
NeventsOld(consolidatedistrOld(10**5,1,'breakPLcut',8, 10, 0.75),'breakPLcut','median')

12.922658073322298

In [22]:
NeventsOld(consolidatedistrOld(10**5,1,'NR',8),'NR','median')

6.2690609388038085

In [50]:
NeventsOld(consolidatedistrOld(10**4,1,'breakPL',8),'breakPL','median')

38.98542234659153

In [58]:
NeventsOld(consolidatedistrOld(10**4,1,'breakPLcut',8),'breakPLcut','median')

17.20017399162604

In [100]:
NeventsOld(consolidatedistrOld(10**3,1,'PL_error',8),'PL_error','median')

45.374568770819394

In [101]:
NeventsOld(consolidatedistrOld(10**4,1,'PL_error',8),'PL_error','median')

46.84905803448296

# Intrinsec event rate NR 

In [355]:
def gendistrOldIntrinsic(N,iteration,SNRth, Tobs, duty, ttmin, ttmax):
    
    N=int(N)
    # Define functions to store intermediate products; I haven't decided on the most convenient format for the final catalogs yet...

    massmin=50*Msol
    massmax=100*Msol

    # Largest horizon redshift; don't waste computing time above this
    zmax = 0.512
    zmin = 0.1
    # Largest comoving distance for sampling
    # largest merger time (yrs)
    tmax = ttmax
    tobs = Tobs
    # ground-based duty cycle
    dutycycle= duty

    data=[]
    for i in range(N):

        #Comoving distance uniform on sphere
        #while True:
        #    cd = np.sum(np.random.uniform(0, cdmax,3)**2)**0.5
        #    if cd<cdmax: break
        # Convert to redshift
        #z = astropy.cosmology.z_at_value(astropy.cosmology.Planck15.comoving_distance,cd*astropy.units.Mpc )
        # Convert to luminosity distance


        z = np.random.uniform(zmin,zmax)
        dl = distL(z)

        # Mass spectrum
        
        index = np.random.choice(len(NRdata))
        m1 = Msol*NRdata[index][0]
        m2 = Msol*NRdata[index][1]
        
        # Sky-location, inclination, polarization, initial phase
        cosiota = random.uniform(-1.,1.)
        psi = random.uniform(0,2*pi)
        costheta=random.uniform(-1.,1.)
        phi=random.uniform(0,2*pi)
        iota=np.arccos(cosiota)
        theta=np.arccos(costheta)
        #phic = np.random.uniform(0,2.*np.pi)
        
        # Merger time
        tmerger=np.random.uniform(ttmin*year,tmax*year)
        
        #fmin=np.random.uniform(1e-5,0.01)
        fmin=findFmin(tmerger,m1,m2)
        Fmax= getfmax(m1*(1+z),m2*(1+z),fmin/(1+z),tobs*year)  #fmax(m1,m2,fmin,tobs*year)
        #Fmax = 1 #Fmax=1
        snr=np.sqrt(dutycycle)*SNR(iota,psi,theta,phi,fmin/(1+z),Fmax,m1*(1+z),m2*(1+z),dl)


        dVcdz = cosmo.comoving_volume(z).value 
        
            
        integralbulk = (tmax-ttmin) * (zmax-zmin) * dVcdz   *np.heaviside(snr-SNRth,0) * (1./(1.+z))

            
        data.append(np.array([m1,m2,z,fmin,integralbulk,snr]))

    return np.array(data).T

def consolidatedistrIntrinsic(Nsingle,iterations,SNRth, Tobs, duty, ttmin, ttmax):

    Nsingle = int(Nsingle)
    iterations = int(iterations)

    Ntot = Nsingle*iterations
    data=[]
    for it in range(0,iterations):
        data.append(gendistrOldIntrinsic(Nsingle,iterations,SNRth, Tobs, duty, ttmin, ttmax))

    data=np.array(data)
    
    m1 = np.concatenate(data[:,0])
    m2 = np.concatenate(data[:,1])
    z = np.concatenate(data[:,2])
    #pdetLIGO = np.concatenate(data[:,3])
    #pdetCE = np.concatenate(data[:,4])
    #tmerger = np.concatenate(data[:,3])
    fmin = np.concatenate(data[:,3])
    #SNR4 = np.concatenate(data[:,6])
    #SNR10 = np.concatenate(data[:,7])
    integralbulk = np.concatenate(data[:,4])
    SNR10 = np.concatenate(data[:,5])

    return m1,m2,z,fmin,integralbulk,SNR10


def NeventsIntrinsic(bigdata):

    # bigdata is the output of consolidatedistr
    m1,m2,z,fmin,integralbulk,SNR10 = bigdata #consolidatedistr(Nsingle,iterations)

    Ntot = len(m1)

    montecarlo_contributions = integralbulk / Ntot

    return np.sum(montecarlo_contributions) * 1e-9



In [340]:
NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 10, 1, 0.001,200))

57.39288307862399

In [332]:
NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 10, 1, 0.1, 50))

51.72455947026308

In [333]:
NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 10, 1, 0.1, 60))

53.76847590478999

# Sample of R and N events 

In [324]:
prova = np.array([np.random.poisson(np.random.gamma(2, 0.596*0.13)*48.354,1)[0] for n in range(100000)])

In [348]:
def givePoisson(Nmed):
    
    arr = np.array([np.random.poisson(np.random.gamma(2, 0.596*0.13)*Nmed,1)[0] for n in range(10000)])
    
    return np.median(arr), np.percentile(arr,95)- np.median(arr), np.percentile(arr,5)- np.median(arr)


In [349]:
# Start with Duty 0.75, 10 obs, Noise 10 pm

NmedToUse = NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 10, 0.75, 0.001,50))

givePoisson(NmedToUse)

(5.0, 11.0, -5.0)

In [350]:
# Start with Duty 0.7, 6 obs, Noise 10 pm

NmedToUse = NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 6, 0.75, 0.001,50))

givePoisson(NmedToUse)

(3.0, 8.049999999999272, -3.0)

In [362]:
# Duty 1, 10 obs, Noise 15 pm

NmedToUse = NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 10, 1, 0.0001,50))

givePoisson(NmedToUse)

(4.0, 9.0, -4.0)

In [357]:
# Duty 1, 6 obs, Noise 15 pm

NmedToUse = NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 6, 1, 0.001,50))

givePoisson(NmedToUse)

(2.0, 6.0, -2.0)

In [358]:
# Duty 0.75, 6 obs, Noise 15 pm

NmedToUse = NeventsIntrinsic(consolidatedistrIntrinsic(10**4,1,8, 6, 0.75, 0.001,50))

givePoisson(NmedToUse)

(2.0, 4.0, -2.0)

In [325]:
t = time.process_time()
np.array([np.random.poisson(np.random.gamma(2, 0.596*0.13)*48.354,1)[0] for n in range(10000)])

elapsed_time = time.process_time() - t
print(elapsed_time)

0.34879400000500027
